In [1]:
import pandas as pd
import requests

In [2]:
df = pd.read_csv("./input/books.csv", error_bad_lines=False)
df.head(30)

b'Skipping line 4012: expected 10 fields, saw 11\nSkipping line 5688: expected 10 fields, saw 11\nSkipping line 7056: expected 10 fields, saw 11\nSkipping line 10601: expected 10 fields, saw 11\nSkipping line 10668: expected 10 fields, saw 11\n'


,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,9780439785969,eng,652,1944099,26249
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,9780439358071,eng,870,1996446,27613
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,9780439554930,eng,320,5629932,70390
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,9780439655484,eng,435,2149872,33964
5,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling-Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,38872,154
6,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.69,0976540606,9780976540601,en-US,152,18,1
7,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,0439827604,9780439827607,eng,3342,27410,820
8,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,0517226952,9780517226957,eng,815,3602,258
9,13,The Ultimate Hitchhiker's Guide to the Galaxy,Douglas Adams,4.38,0345453743,9780345453747,eng,815,240189,3954


In [19]:
def getGenre(isbn):
    url = "https://www.googleapis.com/books/v1/volumes?q=isbn:{}".format(isbn)
    print("Requesting url {}...".format(url))
    res = requests.get(url)
    data = res.json()
    print("done")
    
    return data
#data['ISBN:'+isbn]['details']['genres']


In [39]:
book = getGenre('9780099595816')

Requesting url https://www.googleapis.com/books/v1/volumes?q=isbn:9780099595816...
done


In [40]:
book

{'kind': 'books#volumes',
 'totalItems': 1,
 'items': [{'kind': 'books#volume',
   'id': 'ZY7h1t8IZjUC',
   'etag': 'huJGqffAtFU',
   'selfLink': 'https://www.googleapis.com/books/v1/volumes/ZY7h1t8IZjUC',
   'volumeInfo': {'title': 'Civilwarland in Bad Decline',
    'authors': ['George Saunders'],
    'publisher': 'Random House',
    'publishedDate': '1997',
    'description': "A brilliant collection of stories from the winner of the Folio Prize A collection of six stories and a novella, CIVILWARLAND IN BAD DECLINE introduces an astonishing new voice in American fiction. George Saunders' vision of America's near future is as black and funny as you can get. He takes us on a trip to the shopping malls and theme parks and enviromental hazards that lie just around the chronological corner, introducing us to gang of misfits and losers struggling to survive in an increasingly haywire world. Saunders' dialogue is superb, his plots marvellously bizarre: he is an original and uniquely American

In [41]:
book['items'][0]['volumeInfo']['description']

"A brilliant collection of stories from the winner of the Folio Prize A collection of six stories and a novella, CIVILWARLAND IN BAD DECLINE introduces an astonishing new voice in American fiction. George Saunders' vision of America's near future is as black and funny as you can get. He takes us on a trip to the shopping malls and theme parks and enviromental hazards that lie just around the chronological corner, introducing us to gang of misfits and losers struggling to survive in an increasingly haywire world. Saunders' dialogue is superb, his plots marvellously bizarre: he is an original and uniquely American voice destined to become one of the most important literary discoveries of his generation."

In [42]:
book['items'][0]['volumeInfo']['categories']

['Short stories']